# Evaluation notebook

## Setting up the training and testing dataframes

In [1]:
import pandas as pd

df_train = pd.read_csv("datasets/txt2sql_alerce_train_v3_1.csv")
df_test = pd.read_csv("datasets/txt2sql_alerce_test_v3_1.csv")

In [2]:
# Information about the dataframes
print(f"Shape of training set: {df_train.shape}")
df_train.head()

Shape of training set: (58, 12)


,req_id,request,table_info,external_knowledge,domain_knowledge,gold_query,difficulty,type,nested_type,rephrased_request,rephrased_request_gpt-3.5-turbo-0125_t0.4,rephrased_request_gpt-4o-2024-05-13_t0.2
0,13,Give me all the SNe that were first detected b...,"['object', 'probability']",\r\n-- mjd date for December = 59914.0\r\n-- m...,\r\n-- Super Nova (SNe) is a large explosion t...,"\r\nSELECT\r\n object.oid, probability.clas...",simple,object,none,NaN,NaN,NaN
1,10,Get the object identifiers and probabilities i...,"['probability', 'object']",0,0,"\r\nSELECT\r\n sq1.oid, sq1.probability as SN...",medium,object,simple,NaN,NaN,NaN
2,15,"Get the object identifiers, probabilities in t...","['object', 'probability', 'detection', 'magstat']",\r\n-- mjd date for September 01 = 60188.0\r\n...,\r\n-- A fast riser is defined as an object wh...,"\r\nSELECT\r\n sq.oid, sq.probability, sq.c...",advanced,other,multi,NaN,NaN,NaN
3,4,"Get the object identifier, candidate identifie...","['object', 'probability', 'magstat', 'detection']",\r\n-- mjd date for the start of the year 2019...,0,"\r\nSELECT\r\n sq.oid, sq.fid, sq.dmdt_first,...",advanced,other,multi,NaN,NaN,NaN
4,25,Query objects within 10 degress of the next po...,"['probability', 'object']",\r\n-- mjd date for February 01 = 59976.0\r\n-...,0,"\r\nWITH catalog ( source_id, ra, dec) AS (\r\...",advanced,spatial,simple,NaN,NaN,NaN


In [3]:
print(f"Shape of testing set: {df_test.shape}")
df_test.head()

Shape of testing set: (52, 12)


,req_id,request,table_info,external_knowledge,domain_knowledge,gold_query,difficulty,type,nested_type,rephrased_request,rephrased_request_gpt-3.5-turbo-0125_t0.4,rephrased_request_gpt-4o-2024-05-13_t0.2
0,8,Query all objects that were first classified a...,"['probability', 'object']",\r\n-- mjd date for august 17 = 60173.0\r\n-- ...,NaN,\r\nSELECT\r\n *\r\nFROM\r\n object\r\nI...,simple,object,none,NaN,NaN,NaN
1,27,"Return the oids, meanra, meandec, ndet, firstm...","['probability', 'object']",\r\n-- object.ndet represents the total number...,NaN,"\r\nSELECT\r\n object.oid, object.meanra, o...",simple,object,simple,NaN,NaN,NaN
2,17,Return the objects classified as AGN that show...,"['probability', 'object', 'detection']",\r\n-- firstmjd represents the time of the fir...,NaN,\r\nSELECT\r\n *\r\nFROM\r\n (\r\nSELECT *\r...,advanced,object,tree,NaN,NaN,NaN
3,37,"""Query the top 200 objects classified SNIa acc...",['probability'],NaN,NaN,\r\nSELECT\r\n *\r\nFROM\r\n probability\r...,simple,object,none,NaN,NaN,NaN
4,34,Given this list of oids ['ZTF17aaadpsi' 'ZTF19...,['feature'],\r\n-- feature.name can be 'Amplitude'\r\n-- f...,\r\n-- Multiband_period: Period obtained using...,\r\nSELECT\r\n *\r\nFROM\r\n feature\r\n...,simple,other,none,NaN,NaN,NaN


## Running experiments

In [9]:
# Experiment and save paths
experiment_path = "experiments"
save_path = ""

# Running parameters
ncpus = 2
db_min = 10
n_tries = 3

# Model to use
model = "gpt-4o"

# RAG parameters
max_tokens = 10000
size = 700
overlap = 300
quantity = 10

# Type of pipeline
format = "python"
self_corr = True
rag_pipe = True
direct = False

In [10]:
# Running compare oids
from pipeline.eval import new_compare_oids

new_compare_oids(df_train[:5], 1, model, max_tokens, format, db_min, n_tries, 
                 self_corr, rag_pipe, direct, size, overlap, quantity)

Tables needed: [object, detection]
Difficulty: medium
Resulting python query: 
# Sub-query to get oids of objects first detected between December 1, 2022, and September 1, 2023
sub_query_oids = '''
SELECT
    oid
FROM
    object
WHERE
    firstmjd BETWEEN 59922 AND 60196
'''

# Sub-query to get oids and class_name for SNe with the highest probability
sub_query_probabilities = '''
SELECT
    oid, class_name
FROM
    probability
WHERE
    classifier_name='lc_classifier'
    AND ranking=1
    AND class_name IN ('SNIa', 'SNIbc', 'SNII', 'SLSN')
'''

# Main query to combine the results and get the required information
full_query = f'''
SELECT
    o.oid, o.firstmjd, o.lastmjd, p.class_name
FROM
    object o
INNER JOIN
    ({sub_query_probabilities}) p
ON
    o.oid = p.oid
WHERE
    o.oid IN ({sub_query_oids})
'''


# Sub-query to get oids of objects first detected between December 1, 2022, and September 1, 2023
sub_query_oids = '''
SELECT
    oid
FROM
    object
WHERE
    firstmjd BETWEEN 59

[{'req_id': 13,
  'n_exp': 1,
  'query_diff': 'simple',
  'query_type': 'object',
  'n_rows_gold': 0,
  'n_rows_pred': 0,
  'rows_oid_pred_true': 0,
  'rows_oid_pred_false': 0,
  'rows_oid_gold_true': 0,
  'rows_oid_gold_false': 0,
  'n_cols_gold': 0,
  'n_cols_pred': 0,
  'cols_pred_true': 0,
  'cols_pred_false': 0,
  'cols_gold_true': 0,
  'cols_gold_false': 0,
  'gold_time': 0,
  'pred_time': 0,
  'error_pred': None,
  'are_equal': False,
  'is_oid': False,
  'cols_pred': 0,
  'cols_gold': 0,
  'columns': 1},
 {'req_id': 10,
  'n_exp': 1,
  'query_diff': 'medium',
  'query_type': 'object',
  'n_rows_gold': 0,
  'n_rows_pred': 0,
  'rows_oid_pred_true': 0,
  'rows_oid_pred_false': 0,
  'rows_oid_gold_true': 0,
  'rows_oid_gold_false': 0,
  'n_cols_gold': 0,
  'n_cols_pred': 0,
  'cols_pred_true': 0,
  'cols_pred_false': 0,
  'cols_gold_true': 0,
  'cols_gold_false': 0,
  'gold_time': 0,
  'pred_time': 0,
  'error_pred': None,
  'are_equal': False,
  'is_oid': False,
  'cols_pred': 0,

In [ ]:
# Running the evaluation in training
from pipeline.eval import run_eval_fcn

run_eval_fcn(df_train, experiment_path, save_path, model, max_tokens, format,
             ncpus, db_min, self_corr, rag_pipe, direct, size, overlap, 
             quantity)